# Latitude & Longitude Scraper

**Intall dependencies**

Install selenium terlebih dahulu dengan `!pip install selenium`

In [ ]:
#Run this code to install selenium
!pip install selenium

Run cell di bawah ini untuk import beberapa library yang akan digunakan 

In [ ]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from tqdm import tqdm_notebook as tqdmn
import warnings

warnings.filterwarnings("ignore")

**Setup**

1. Pastikan chromedriver berada dalam satu folder dengan file notebook ini, jika tidak maka isi variabel `exec_path` pada baris kedua kode di bawah ini dengan filepath yang sesuai dengan letak file `chromedriver.exe`
2. Pastikan chrome versi 86.0.4240.75, jika tidak bisa update chrome dahulu agar compatible dengan chromedrivernya
3. Jika sudah sesuai maka run code di bawah ini 

In [ ]:
#Parameters

#Batas bawah dan batas atas waktu tunggu per iterasi
#Contoh: delay per iterasi 2-5 detik, maka set delay_lower_bound = 2 dan delay_upper_bound = 5
delay_lower_bound = 2
delay_upper_bound = 5

#Tambahan waktu (detik) untuk scraper mengekstrak koordinat
implicit_wait = 1

#Jumlah row per partisi
part_size = 100

In [ ]:
def scrape_coordinates(file, sheetname, address_col, output):
    exec_path = 'chromedriver.exe'
    #Read input file
    try:
        if file.rsplit('.', 1)[-1] == 'xlsx':
            df_all = pd.read_excel(file, sheet_name= sheetname)
        elif file.rsplit('.', 1)[-1] == 'csv':
            df_all = pd.read_csv(file)

    except:
        raise Exception("File input error")
        
    driver = webdriver.Chrome(exec_path)
    
    #Partition
    n_part = int(np.floor(df_all.shape[0]/part_size) + 1)
    df_result = pd.DataFrame()
    
    for i in range(n_part):
        df = df_all.copy()
        if i == n_part:
            df = df_all.copy().iloc[i*part_size:df_all.shape[0], :]
        else:
            df = df_all.copy().iloc[(i*part_size):(part_size*(i+1)), :]
    
        #Scraping coordinates
        url_with_coordinates = []
        lat_scrape = []
        long_scrape = []
        colnames = address_col
        for item in tqdmn(df[colnames]):
            url = 'https://www.google.com/maps/search/' + str(item)
            delay = np.random.rand()*(delay_upper_bound - delay_lower_bound) + delay_lower_bound
            time.sleep(delay)
            driver.get(url)
            driver.implicitly_wait(implicit_wait)
            url_with_coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))
        #driver.close()

        df['url_with_coordinates'] = url_with_coordinates
        for url in df['url_with_coordinates']:
            try:
                lat_scrape.append(url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0])
            except:
                lat_scrape.append('NA')
            try:
                long_scrape.append(url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1])
            except:
                long_scrape.append('NA')

        df['lat_scrape'] = lat_scrape
        df['long_scrape'] = long_scrape
        
        #Export hasil ke dalam excel bernama 'HasilScraping.xlsx'
        df_result = pd.concat([df_result, df], axis = 0)
        df_result.to_excel(output, index = False)
        
        #print
        print('Partisi ke-{} dari {} selesai'.format(i+1, n_part))
    
    return df_result

**Menjalankan Fungsi**
1. Untuk melakukan scraping, dapat dilakukan dengan menjalankan fungsi `scrape_coordinates` dengan 3 argumen, yaitu file, sheetname, dan address_col
2. `file` diisi dengan lokasi input file alamat yang ingin dilakukan scraping. Untuk memudahkan, letakkan file input dalam satu folder dengan notebook ini. Pastikan file input berekstensi `.xlsx` atau `.csv`, jika tidak maka akan menghasilkan error.
3. `sheetname` merupakan nama sheet pada excel yang berisi informasi alamat yang ingin discraping
4. `address_col` merupakan nama kolom yang berisi alamat-alamat yang ingin discraping
5. `output` merupakan nama file hasil export (excel)

Misalkan alamat yang ingin discrape berada pada file bernama `dummy1.xlsx` pada `Sheet1` dan pada kolom `Address` serta hasil akan diexport ke dalam excel bernama `HasilScraping.xlsx`, maka jalankan fungsi `scrape_coordinates(file = 'dummy1.xlsx', sheetname = 'Sheet1', address_col = 'Address', output = 'HasilScraping.xlsx')`

Hasil scraping akan diinput pada kolom `lat_scrape` dan `long_scrape` url hasil scraping yang mengandung informasi koordinat diinput ke dalam kolom `url_with_coordinates` (kolom ini dapat dihapus jika memang tidak diperlukan)

In [ ]:
#Contoh scraping dan memasukkannya pada variabel df_scrape serta hasil di save pada excel 'HasilScraping.xlsx'
df_scrape = scrape_coordinates(file = 'dummy1.xlsx', 
                               sheetname = 'Sheet1', 
                               address_col = 'Address', 
                               output = 'HasilScraping.xlsx')

**Koordinat yang tidak bisa ditemukan**

Untuk koordinat yang tidak bisa diperoleh melalui scraping akan ditandai dengan `'NA'`. Dalam beberapa kasus, koordinat tidak bisa ditemukan salah satunya karena alamat yang diberikan tidak bisa ditemukan melalui google maps